## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jan. 12th Jan. 19th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,January-13-2025,Condo/Co-op,1881 79th St Cswy #1403,North Bay Village,FL,33141.0,370000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Bay-Village/18...,MARMLS,A11706067,N,Y,25.848811,-80.146478


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [9]:
df2 = df

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-01-15    31
2025-01-13    24
2025-01-17    18
2025-01-14    17
2025-01-16    16
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
76,PAST SALE,2025-01-17,Condo/Co-op,360 Ocean Dr Unit LPH2S,Key Biscayne,FL,33149.0,15200000.0,4.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11652970,N,Y,25.694584,-80.157715
35,PAST SALE,2025-01-16,Condo/Co-op,1000 Biscayne Blvd #1902,Miami,FL,33132.0,5800000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11694327,N,Y,25.784171,-80.190078
94,PAST SALE,2025-01-15,Condo/Co-op,4701 N Meridian Ave #222,Miami Beach,FL,33140.0,5125000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11612834,N,Y,25.822813,-80.130733
49,PAST SALE,2025-01-15,Condo/Co-op,1300 Monad Ter Unit 9D,Miami Beach,FL,33139.0,4003777.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11671666,N,Y,25.784162,-80.142621
99,PAST SALE,2025-01-15,Condo/Co-op,15701 Collins Ave #3604,Sunny Isles Beach,FL,33160.0,3725000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11539203,N,Y,25.921944,-80.121716
71,PAST SALE,2025-01-16,Condo/Co-op,1228 Anastasia Ave #203,Coral Gables,FL,33134.0,3000000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/1228-An...,MARMLS,A11641232,N,Y,25.741984,-80.281254
82,PAST SALE,2025-01-13,Condo/Co-op,1451 Brickell Ave #4101,Miami,FL,33131.0,2300000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11646789,N,Y,25.758141,-80.192431
74,PAST SALE,2025-01-13,Condo/Co-op,1000 Brickell Plz #2102,Miami,FL,33131.0,2150000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11661324,N,Y,25.764264,-80.192725
51,PAST SALE,2025-01-14,Condo/Co-op,125 Ocean Dr Unit F0203,Miami Beach,FL,33139.0,2075000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,MARMLS,A11668789,N,Y,25.770080,-80.132744
77,PAST SALE,2025-01-16,Condo/Co-op,17475 E Collins Ave #804,Sunny Isles Beach,FL,33160.0,1700000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11648224,N,Y,25.937942,-80.120683


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-01-17,Condo/Co-op,360 Ocean Dr Unit LPH2S,Key Biscayne,FL,33149.0,15200000.0,4.0,6.5,...,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Juliana Savoia,Coldwell Banker Realty,None,None,Florencia Capiel De Moni,BHHS EWM Realty,None,None
1,PAST SALE,2025-01-16,Condo/Co-op,1000 Biscayne Blvd #1902,Miami,FL,33132.0,5800000.0,4.0,5.5,...,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,David Siddons,Douglas Elliman,None,None,Raymond De Leon,One Sotheby's International Realty,None,None
2,PAST SALE,2025-01-15,Condo/Co-op,4701 N Meridian Ave #222,Miami Beach,FL,33140.0,5125000.0,2.0,2.5,...,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Marko Gojanovic,One Sotheby's International Re,None,None
3,PAST SALE,2025-01-15,Condo/Co-op,1300 Monad Ter Unit 9D,Miami Beach,FL,33139.0,4003777.0,3.0,3.5,...,-80.142621,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Jared Robins,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Richard Teed,The Agency Florida LLC,None,None
4,PAST SALE,2025-01-15,Condo/Co-op,15701 Collins Ave #3604,Sunny Isles Beach,FL,33160.0,3725000.0,2.0,2.5,...,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Lana Bell,Douglas Elliman,None,None,Tomi Rose,Aston Rose Sports And Entertai,None,None
5,PAST SALE,2025-01-16,Condo/Co-op,1228 Anastasia Ave #203,Coral Gables,FL,33134.0,3000000.0,3.0,3.0,...,-80.281254,https://www.redfin.com/FL/Coral-Gables/1228-An...,Rosi Borroto,One Sotheby's International Realty,Maria Longo,One Sotheby's International Realty,Brandon Janvion,EXP Realty LLC,None,None
6,PAST SALE,2025-01-13,Condo/Co-op,1451 Brickell Ave #4101,Miami,FL,33131.0,2300000.0,2.0,3.0,...,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Adam Redolfi,Alure Capital,None,None,Virginia Pinto,EXP Realty LLC,None,None
7,PAST SALE,2025-01-13,Condo/Co-op,1000 Brickell Plz #2102,Miami,FL,33131.0,2150000.0,3.0,3.5,...,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Monica Buitrago,"Cornerstone Intl. Realty, LLC",None,None,Vilma Datorre,Datorre Realty Group,None,None
8,PAST SALE,2025-01-14,Condo/Co-op,125 Ocean Dr Unit F0203,Miami Beach,FL,33139.0,2075000.0,2.0,2.5,...,-80.132744,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,Haydee Mederos,Fortune Christie's International Real Estate,Sofi Ojeda,Fortune Christie's International Real Estate,Richard Corrales,Silverleaf Realty Group,None,None
9,PAST SALE,2025-01-16,Condo/Co-op,17475 E Collins Ave #804,Sunny Isles Beach,FL,33160.0,1700000.0,2.0,2.0,...,-80.120683,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Michelle Judd,Dezer Platinum Realty LLC,None,None,Martin Zelikson,"Concrete Realty, LLC.",None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

106


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$899,049


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$602


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$95,299,146


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
93
Input Previous Week Condo Average Sales Price:
665359
Input Previous Week Condo Average PSF:
503
Input Previous Week Condo Sales total (ex: 198_000_000)
61_900_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-17,Condo/Co-op,360 Ocean Dr Unit LPH2S,Key Biscayne,FL,33149.0,15200000.0,4.0,6.5,OCEANA KEY BISCAYNE CONDO,4762.0,NaN,2014.0,NaN,3192.0,7175.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11652970,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Juliana Savoia,Coldwell Banker Realty,None,None,Florencia Capiel De Moni,BHHS EWM Realty,None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-17,Condo/Co-op,360 Ocean Dr Unit LPH2S,Key Biscayne,FL,33149.0,15200000.0,4.0,6.5,OCEANA KEY BISCAYNE CONDO,4762.0,NaN,2014.0,NaN,3192.0,7175.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11652970,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Juliana Savoia,Coldwell Banker Realty,None,None,Florencia Capiel De Moni,BHHS EWM Realty,None,None,1,orange
1,PAST SALE,2025-01-16,Condo/Co-op,1000 Biscayne Blvd #1902,Miami,FL,33132.0,5800000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1261.0,8380.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11694327,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,David Siddons,Douglas Elliman,None,None,Raymond De Leon,One Sotheby's International Realty,None,None,2,blue
2,PAST SALE,2025-01-15,Condo/Co-op,4701 N Meridian Ave #222,Miami Beach,FL,33140.0,5125000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2674.0,NaN,2019.0,NaN,1917.0,5356.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11612834,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Marko Gojanovic,One Sotheby's International Re,None,None,3,blue
3,PAST SALE,2025-01-15,Condo/Co-op,1300 Monad Ter Unit 9D,Miami Beach,FL,33139.0,4003777.0,3.0,3.5,ALTON BEACH REALTY CO SUB,2496.0,NaN,2021.0,NaN,1604.0,5818.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11671666,N,Y,25.784162,-80.142621,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Jared Robins,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Richard Teed,The Agency Florida LLC,None,None,4,blue
4,PAST SALE,2025-01-15,Condo/Co-op,15701 Collins Ave #3604,Sunny Isles Beach,FL,33160.0,3725000.0,2.0,2.5,RITZ CARLTON,1571.0,NaN,2018.0,NaN,2371.0,2675.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11539203,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Lana Bell,Douglas Elliman,None,None,Tomi Rose,Aston Rose Sports And Entertai,None,None,5,blue
5,PAST SALE,2025-01-16,Condo/Co-op,1228 Anastasia Ave #203,Coral Gables,FL,33134.0,3000000.0,3.0,3.0,VILLA BILTMORE,2791.0,NaN,2021.0,NaN,1075.0,4954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/1228-An...,MARMLS,A11641232,N,Y,25.741984,-80.281254,https://www.redfin.com/FL/Coral-Gables/1228-An...,Rosi Borroto,One Sotheby's International Realty,Maria Longo,One Sotheby's International Realty,Brandon Janvion,EXP Realty LLC,None,None,6,blue
6,PAST SALE,2025-01-13,Condo/Co-op,1451 Brickell Ave #4101,Miami,FL,33131.0,2300000.0,2.0,3.0,ECHO BRICKELL CONDO,1627.0,NaN,2017.0,NaN,1414.0,3785.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11646789,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Adam Redolfi,Alure Capital,None,None,Virginia Pinto,EXP Realty LLC,None,None,7,blue
7,PAST SALE,2025-01-13,Condo/Co-op,1000 Brickell Plz #2102,Miami,FL,33131.0,2150000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1909.0,NaN,2019.0,NaN,1126.0,2947.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11661324,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Monica Buitrago,"Cornerstone Intl. Realty, LLC",None,None,Vilma Datorre,Datorre Realty Group,None,None,8,blue
8,PAST SALE,2025-01-14,Condo/Co-op,125 Ocean Dr Unit F0203,Miami Beach,FL,33139.0,2075000.0,2.0,2.5,OCEAN HOUSE CONDO,1370.0,NaN,

In [32]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [33]:
m.save('index.html')

## Data snagger

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-17,Condo/Co-op,360 Ocean Dr Unit LPH2S,Key Biscayne,FL,33149.0,15200000.0,4.0,6.5,OCEANA KEY BISCAYNE CONDO,4762.0,NaN,2014.0,NaN,3192.0,7175.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11652970,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Juliana Savoia,Coldwell Banker Realty,None,None,Florencia Capiel De Moni,BHHS EWM Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_01202024


# CREATE TEMPLATE 

In [37]:
muni_set = set(df_top_ten['CITY'])

In [38]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [39]:
df_top_ten.reset_index(inplace=True,drop=True)

In [40]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [41]:
top_sale

'OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit LPH2S in Key Biscayne'

In [42]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [43]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-17,Condo/Co-op,360 Ocean Dr Unit LPH2S,Key Biscayne,FL,33149.0,15200000.0,4.0,6.5,OCEANA KEY BISCAYNE CONDO,4762.0,NaN,2014.0,NaN,3192.0,7175.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11652970,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Juliana Savoia,Coldwell Banker Realty,None,None,Florencia Capiel De Moni,BHHS EWM Realty,None,None,1,orange,OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit...
1,PAST SALE,2025-01-16,Condo/Co-op,1000 Biscayne Blvd #1902,Miami,FL,33132.0,5800000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1261.0,8380.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11694327,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,David Siddons,Douglas Elliman,None,None,Raymond De Leon,One Sotheby's International Realty,None,None,2,blue,ONE THOUSAND MUSEUM CONDO at 1000 Biscayne Blv...
2,PAST SALE,2025-01-15,Condo/Co-op,4701 N Meridian Ave #222,Miami Beach,FL,33140.0,5125000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2674.0,NaN,2019.0,NaN,1917.0,5356.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11612834,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Marko Gojanovic,One Sotheby's International Re,None,None,3,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
3,PAST SALE,2025-01-15,Condo/Co-op,1300 Monad Ter Unit 9D,Miami Beach,FL,33139.0,4003777.0,3.0,3.5,ALTON BEACH REALTY CO SUB,2496.0,NaN,2021.0,NaN,1604.0,5818.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11671666,N,Y,25.784162,-80.142621,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Jared Robins,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Richard Teed,The Agency Florida LLC,None,None,4,blue,ALTON BEACH REALTY CO SUB at 1300 Monad Ter Un...
4,PAST SALE,2025-01-15,Condo/Co-op,15701 Collins Ave #3604,Sunny Isles Beach,FL,33160.0,3725000.0,2.0,2.5,RITZ CARLTON,1571.0,NaN,2018.0,NaN,2371.0,2675.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11539203,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Lana Bell,Douglas Elliman,None,None,Tomi Rose,Aston Rose Sports And Entertai,None,None,5,blue,RITZ CARLTON at 15701 Collins Ave #3604 in Sun...
5,PAST SALE,2025-01-16,Condo/Co-op,1228 Anastasia Ave #203,Coral Gables,FL,33134.0,3000000.0,3.0,3.0,VILLA BILTMORE,2791.0,NaN,2021.0,NaN,1075.0,4954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/1228-An...,MARMLS,A11641232,N,Y,25.741984,-80.281254,https://www.redfin.com/FL/Coral-Gables/1228-An...,Rosi Borroto,One Sotheby's International Realty,Maria Longo,One Sotheby's International Realty,Brandon Janvion,EXP Realty LLC,None,None,6,blue,VILLA BILTMORE at 1228 Anastasia Ave #203 in C...
6,PAST SALE,2025-01-13,Condo/Co-op,1451 Brickell Ave #4101,Miami,FL,33131.0,2300000.0,2.0,3.0,ECHO BRICKELL CONDO,1627.0,NaN,2017.0,NaN,1414.0,3785.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11646789,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Adam Redolfi,Alure Capital,None,None,Virginia Pinto,EXP Realty LLC,None,None,7,blue,ECHO BRICKELL CONDO at 1451 Brickell Ave #4101...
7,PAST SALE,2025-01-13,Condo/Co-op,1000 Brickell Plz #2102,Miami,FL,33131.0,2150000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1909.0,NaN,2019.0,NaN,1126.0,2947.0,Sold,NaN,NaN,https://www.redfin.co

In [45]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: OCEANA KEY BISCAYNE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,700,000 to $15,200,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Key Biscayne, Sunny Isles Beach, Miami, Coral Gables

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 106 condo sales totaling $95,299,146 million from Jan. 12th to Jan. 19th. The previous week, brokers closed 93 condo sales totaling $61,900,000.

Last week’s units sold for an average of $899,049, higher than the $665,359 average price from the p

In [46]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [47]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-17,Condo/Co-op,360 Ocean Dr Unit LPH2S,Key Biscayne,FL,33149.0,15200000.0,4.0,6.5,OCEANA KEY BISCAYNE CONDO,4762.0,NaN,2014.0,NaN,3192.0,7175.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11652970,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Juliana Savoia,Coldwell Banker Realty,None,None,Florencia Capiel De Moni,BHHS EWM Realty,None,None,1,orange,OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit...
1,PAST SALE,2025-01-16,Condo/Co-op,1000 Biscayne Blvd #1902,Miami,FL,33132.0,5800000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1261.0,8380.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11694327,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,David Siddons,Douglas Elliman,None,None,Raymond De Leon,One Sotheby's International Realty,None,None,2,blue,ONE THOUSAND MUSEUM CONDO at 1000 Biscayne Blv...
2,PAST SALE,2025-01-15,Condo/Co-op,4701 N Meridian Ave #222,Miami Beach,FL,33140.0,5125000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2674.0,NaN,2019.0,NaN,1917.0,5356.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11612834,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Marko Gojanovic,One Sotheby's International Re,None,None,3,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
3,PAST SALE,2025-01-15,Condo/Co-op,1300 Monad Ter Unit 9D,Miami Beach,FL,33139.0,4003777.0,3.0,3.5,ALTON BEACH REALTY CO SUB,2496.0,NaN,2021.0,NaN,1604.0,5818.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11671666,N,Y,25.784162,-80.142621,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Jared Robins,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Richard Teed,The Agency Florida LLC,None,None,4,blue,ALTON BEACH REALTY CO SUB at 1300 Monad Ter Un...
4,PAST SALE,2025-01-15,Condo/Co-op,15701 Collins Ave #3604,Sunny Isles Beach,FL,33160.0,3725000.0,2.0,2.5,RITZ CARLTON,1571.0,NaN,2018.0,NaN,2371.0,2675.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11539203,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Lana Bell,Douglas Elliman,None,None,Tomi Rose,Aston Rose Sports And Entertai,None,None,5,blue,RITZ CARLTON at 15701 Collins Ave #3604 in Sun...
5,PAST SALE,2025-01-16,Condo/Co-op,1228 Anastasia Ave #203,Coral Gables,FL,33134.0,3000000.0,3.0,3.0,VILLA BILTMORE,2791.0,NaN,2021.0,NaN,1075.0,4954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/1228-An...,MARMLS,A11641232,N,Y,25.741984,-80.281254,https://www.redfin.com/FL/Coral-Gables/1228-An...,Rosi Borroto,One Sotheby's International Realty,Maria Longo,One Sotheby's International Realty,Brandon Janvion,EXP Realty LLC,None,None,6,blue,VILLA BILTMORE at 1228 Anastasia Ave #203 in C...
6,PAST SALE,2025-01-13,Condo/Co-op,1451 Brickell Ave #4101,Miami,FL,33131.0,2300000.0,2.0,3.0,ECHO BRICKELL CONDO,1627.0,NaN,2017.0,NaN,1414.0,3785.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11646789,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Adam Redolfi,Alure Capital,None,None,Virginia Pinto,EXP Realty LLC,None,None,7,blue,ECHO BRICKELL CONDO at 1451 Brickell Ave #4101...
7,PAST SALE,2025-01-13,Condo/Co-op,1000 Brickell Plz #2102,Miami,FL,33131.0,2150000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1909.0,NaN,2019.0,NaN,1126.0,2947.0,Sold,NaN,NaN,https://www.redfin.co

In [50]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: OCEANA KEY BISCAYNE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,700,000 to $15,200,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Key Biscayne, Sunny Isles Beach, Miami, Coral Gables

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 106 condo sales totaling $95,299,146 million from Jan. 12th to Jan. 19th. The previous week, brokers closed 93 condo sales totaling $61,900,000.

Last week’s units sold for an average of $899,049, higher than the $665,359 average price from the p

In [51]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Key-Biscayne/360-Ocean-Dr-33149/unit-LPH2S/home/192405631


In [52]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami/1000-Biscayne-Blvd-33132/unit-1902/home/113531469


In [53]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17475-Collins-Ave-33160/unit-804/home/108547316


In [54]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/125-Ocean-Dr-33139/unit-F0203/home/43474296


In [55]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Key-Biscayne/360-Ocean-Dr-33149/unit-LPH2S/home/192405631


In [56]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Coral-Gables/1228-Anastasia-Ave-33134/unit-203/home/148721592


In [57]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-17,Condo/Co-op,360 Ocean Dr Unit LPH2S,Key Biscayne,FL,33149.0,15200000.0,4.0,6.5,OCEANA KEY BISCAYNE CONDO,4762.0,NaN,2014.0,NaN,3192.0,7175.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11652970,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Juliana Savoia,Coldwell Banker Realty,None,None,Florencia Capiel De Moni,BHHS EWM Realty,None,None,1,orange,OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit...
1,PAST SALE,2025-01-16,Condo/Co-op,1000 Biscayne Blvd #1902,Miami,FL,33132.0,5800000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1261.0,8380.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11694327,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,David Siddons,Douglas Elliman,None,None,Raymond De Leon,One Sotheby's International Realty,None,None,2,blue,ONE THOUSAND MUSEUM CONDO at 1000 Biscayne Blv...
2,PAST SALE,2025-01-15,Condo/Co-op,4701 N Meridian Ave #222,Miami Beach,FL,33140.0,5125000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,2674.0,NaN,2019.0,NaN,1917.0,5356.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11612834,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Marko Gojanovic,One Sotheby's International Re,None,None,3,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
3,PAST SALE,2025-01-15,Condo/Co-op,1300 Monad Ter Unit 9D,Miami Beach,FL,33139.0,4003777.0,3.0,3.5,ALTON BEACH REALTY CO SUB,2496.0,NaN,2021.0,NaN,1604.0,5818.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,MARMLS,A11671666,N,Y,25.784162,-80.142621,https://www.redfin.com/FL/Miami-Beach/1300-Mon...,Jared Robins,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Richard Teed,The Agency Florida LLC,None,None,4,blue,ALTON BEACH REALTY CO SUB at 1300 Monad Ter Un...
4,PAST SALE,2025-01-15,Condo/Co-op,15701 Collins Ave #3604,Sunny Isles Beach,FL,33160.0,3725000.0,2.0,2.5,RITZ CARLTON,1571.0,NaN,2018.0,NaN,2371.0,2675.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11539203,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Lana Bell,Douglas Elliman,None,None,Tomi Rose,Aston Rose Sports And Entertai,None,None,5,blue,RITZ CARLTON at 15701 Collins Ave #3604 in Sun...
5,PAST SALE,2025-01-16,Condo/Co-op,1228 Anastasia Ave #203,Coral Gables,FL,33134.0,3000000.0,3.0,3.0,VILLA BILTMORE,2791.0,NaN,2021.0,NaN,1075.0,4954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/1228-An...,MARMLS,A11641232,N,Y,25.741984,-80.281254,https://www.redfin.com/FL/Coral-Gables/1228-An...,Rosi Borroto,One Sotheby's International Realty,Maria Longo,One Sotheby's International Realty,Brandon Janvion,EXP Realty LLC,None,None,6,blue,VILLA BILTMORE at 1228 Anastasia Ave #203 in C...
6,PAST SALE,2025-01-13,Condo/Co-op,1451 Brickell Ave #4101,Miami,FL,33131.0,2300000.0,2.0,3.0,ECHO BRICKELL CONDO,1627.0,NaN,2017.0,NaN,1414.0,3785.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11646789,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Adam Redolfi,Alure Capital,None,None,Virginia Pinto,EXP Realty LLC,None,None,7,blue,ECHO BRICKELL CONDO at 1451 Brickell Ave #4101...
7,PAST SALE,2025-01-13,Condo/Co-op,1000 Brickell Plz #2102,Miami,FL,33131.0,2150000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1909.0,NaN,2019.0,NaN,1126.0,2947.0,Sold,NaN,NaN,https://www.redfin.co

## Time on Market Calculator

In [59]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 4, 9) ## List (Earlier) date
date2 = datetime(2025, 1, 16) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

282


## Clean CSV for Datawrapper Chart

In [60]:
chart_df = df_top_ten

In [61]:
chart_df = chart_df.fillna(" ")

In [62]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [63]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [64]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [65]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [66]:
chart_df['$/SQUARE FEET'].astype(float)

0    3192.0
1    1261.0
2    1917.0
3    1604.0
4    2371.0
5    1075.0
6    1414.0
7    1126.0
8    1515.0
9    1160.0
Name: $/SQUARE FEET, dtype: float64

In [67]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [68]:
csv_date_string = today.strftime("%m_%d_%Y")

In [69]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [70]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-17,360 Ocean Dr Unit LPH2S in Key Biscayne,"$15,200,000",4,6.5,"4,762",2014,"$3,192",MARMLS,A11652970,Juliana Savoia Coldwell Banker Realty,Florencia Capiel De Moni BHHS EWM Realty
1,2025-01-16,1000 Biscayne Blvd #1902 in Miami,"$5,800,000",4,5.5,"4,600",2019,"$1,261",MARMLS,A11694327,David Siddons Douglas Elliman,Raymond De Leon One Sotheby's International Re...
2,2025-01-15,4701 N Meridian Ave #222 in Miami Beach,"$5,125,000",2,2.5,"2,674",2019,"$1,917",MARMLS,A11612834,Ivan Madera Fortune Christie's International R...,Marko Gojanovic One Sotheby's International Re...
3,2025-01-15,1300 Monad Ter Unit 9D in Miami Beach,"$4,003,777",3,3.5,"2,496",2021,"$1,604",MARMLS,A11671666,Jared Robins One Sotheby's International Re Re...,Richard Teed The Agency Florida LLC
4,2025-01-15,15701 Collins Ave #3604 in Sunny Isles Beach,"$3,725,000",2,2.5,"1,571",2018,"$2,371",MARMLS,A11539203,Lana Bell Douglas Elliman,Tomi Rose Aston Rose Sports And Entertai
5,2025-01-16,1228 Anastasia Ave #203 in Coral Gables,"$3,000,000",3,3.0,"2,791",2021,"$1,075",MARMLS,A11641232,Rosi Borroto One Sotheby's International Realt...,Brandon Janvion EXP Realty LLC
6,2025-01-13,1451 Brickell Ave #4101 in Miami,"$2,300,000",2,3.0,"1,627",2017,"$1,414",MARMLS,A11646789,Adam Redolfi Alure Capital,Virginia Pinto EXP Realty LLC
7,2025-01-13,1000 Brickell Plz #2102 in Miami,"$2,150,000",3,3.5,"1,909",2019,"$1,126",MARMLS,A11661324,"Monica Buitrago Cornerstone Intl. Realty, LLC",Vilma Datorre Datorre Realty Group
8,2025-01-14,125 Ocean Dr Unit F0203 in Miami Beach,"$2,075,000",2,2.5,"1,370",2009,"$1,515",MARMLS,A11668789,Haydee Mederos Fortune Christie's Internationa...,Richard Corrales Silverleaf Realty Group
9,2025-01-16,17475 E Collins Ave #804 in Sunny Isles Beach,"$1,700,000",2,2.0,"1,465",2016,"$1,160",MARMLS,A11648224,Michelle Judd Dezer Platinum Realty LLC,"Martin Zelikson Concrete Realty, LLC."
